# Explore Kubeflow Katib Hyperparamter Optimization API

In [7]:
!pip install git+https://github.com/kubeflow/katib.git#subdirectory=sdk/python/v1beta1
!pip install git+https://github.com/kubeflow/training-operator.git#subdirectory=sdk/python

  Cloning https://github.com/kubeflow/katib.git to /private/var/folders/_z/49hgcx6x4db7pjpfzjcphlcm0000gn/T/pip-req-build-a6ajaxks
  Running command git clone --filter=blob:none --quiet https://github.com/kubeflow/katib.git /private/var/folders/_z/49hgcx6x4db7pjpfzjcphlcm0000gn/T/pip-req-build-a6ajaxks
  Resolved https://github.com/kubeflow/katib.git to commit 93bee4dc25529a92320cf33634bb6920013c8281
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/kubeflow/training-operator.git to /private/var/folders/_z/49hgcx6x4db7pjpfzjcphlcm0000gn/T/pip-req-build-95zfygeb
  Running command git clone --filter=blob:none --quiet https://github.com/kubeflow/training-operator.git /private/var/folders/_z/49hgcx6x4db7pjpfzjcphlcm0000gn/T/pip-req-build-95zfygeb
  Resolved https://github.com/kubeflow/training-operator.git to commit 1dfa40c12516fc9eb2ce12c5ef52da7d46670457
  Preparing metadata (setup.py) ... done
  Created wheel for kubeflow-training: filename=kubeflow_training-1.8.1-py

In [2]:
!pip install transformers peft

  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached huggingface_hub-0.27.1-py3-none-any.whl.metadata (13 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached safetensors-0.5.2-cp38-abi3-macosx_11_0_arm64.whl.metadata (3.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 20.3 MB/s eta 0:00:00a 0:00:01
Using cached huggingface_hub-0.27.1-py3-none-any.whl (450 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 12.7 MB/s eta 0:00:0000:0100:01
Using cached safetensors-0.5.2-cp38-abi3-macosx_11_0_arm64.whl (408 kB)
Using cached tokenizers-0.21.0-cp39-abi3-macosx_11_0_arm64.whl (2.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 17.4 MB/

In [51]:
# from kubeflow.katib.types.trainer_resources import TrainerResources
# 
# 
# resources_per_trial = TrainerResources(
#     num_workers=4,                    # Number of distributed workers
#     num_procs_per_worker=2,           # Processes per worker
#     resources_per_worker={            # Resource allocation per worker
#         "gpu": 2,                     # Number of GPUs
#         "cpu": 5,                     # Number of CPUs
#         "memory": "10G",              # Memory allocation
#     },
# )


In [52]:
import kubeflow.katib as katib
from kubeflow.katib import KatibClient

from transformers import AutoModelForSequenceClassification, TrainingArguments
from peft import LoraConfig

from kubeflow.storage_initializer.hugging_face import (
    HuggingFaceModelParams,
    HuggingFaceDatasetParams,
    HuggingFaceTrainerParams,
)

In [56]:
hf_model = HuggingFaceModelParams(
    model_uri = "hf://meta-llama/Llama-3.2-1B",
    transformer_type = AutoModelForSequenceClassification,
)

# Train the model on 1000 movie reviews from imdb
# https://huggingface.co/datasets/stanfordnlp/imdb
hf_dataset = HuggingFaceDatasetParams(
    repo_id = "imdb",
    split = "train[:1000]",
)

hf_tuning_parameters = HuggingFaceTrainerParams(
    training_parameters = TrainingArguments(
        output_dir = "results",
        save_strategy = "epoch",
        hub_strategy="all_checkpoints",
        learning_rate = 1e-05, #katib.search.double(min=1e-05, max=5e-05),
        num_train_epochs=3,
    ),
    # Set LoRA config to reduce number of trainable model parameters.
    lora_config = LoraConfig(
        r = 1, # katib.search.int(min=8, max=32),
        lora_alpha = 8,
        lora_dropout = 0.1,
        bias = "none",
    ),
)

In [57]:
cl = KatibClient(namespace="kubeflow")

In [58]:
# Fine-tuning for Binary Classification
exp_name = "testllm"
cl.tune(
    name = exp_name,
    model_provider_parameters = hf_model,
    dataset_provider_parameters = hf_dataset,
    trainer_parameters = hf_tuning_parameters,
    objective_metric_name = "train_loss",
    objective_type = "minimize",
    algorithm_name = "random",
    # max_trial_count = 10,
    # parallel_trial_count = 2,
    # resources_per_trial={
    #     "gpu": "2",
    #     "cpu": "4",
    #     "memory": "10G",
    # },
)

cl.wait_for_experiment_condition(name=exp_name)

# Get the best hyperparameters.
print(cl.get_optimal_hyperparameters(exp_name))

Thank you for using `tune` API for LLM hyperparameter optimization. This feature is in the alpha stage. Kubeflow community is looking for your feedback. Please share your experience via #kubeflow-katib Slack channel or the Kubeflow Katib GitHub.
PVC 'testllm' already exists in namespace kubeflow.


TypeError: Object of type LoraRuntimeConfig is not JSON serializable